In [1]:
from import_library import *
from model_stack import model_list

In [2]:
print("CPU count: {}".format(multiprocessing.cpu_count()))

CPU count: 8


In [3]:
from itertools import compress
# feature selection process
rank_threshold = 80
def feature_filter(patient_value, load_path = None,outcome_name = None, data_type_ = 'mean'):
    feature_rank_df = pd.read_csv('{}/{}_{}_{}.csv'.format(load_path, outcome_name,data_type_, 'alff'), index_col=False)
    filter_feature_list = [feature_rank_df['rank'].values<=rank_threshold][0]
    feature_columns = patient_value.columns.tolist()
    patient_value = patient_value[list(compress(feature_columns, filter_feature_list))]

    feature_columns = list(compress(feature_columns, filter_feature_list))
    return patient_value, feature_columns

In [4]:
data_type ='mix' #mat, csv, mix
feature_selection_df_savepath = './feature_selection_result'
# dataloader
clinical_data = pd.read_csv('./data/clinical data/clinical_data_2.csv')[['subj', 'AD8主觀認知障礙[0-1,>=2]', 'MOCA客觀認知測驗分數[<=23,>=24]', 'HADS_A焦慮程度', 'HADS_D憂鬱程度']]
raw_data_path = './data/mdALFF matrix excel/' #raw_data_path = './data/image data/dFC matrix matlab/'
if data_type!='mix':
    next_path =f'mdALFF_{data_type}.csv' #next_path ='C0005FC_z.mat't'
else:
    next_path = ['mdALFF_mean.csv', 'mdALFF_var.csv']
def loader_(data_path = None ,mat_dtype='FCM_mean', outcome_name = None):
    if data_type =='mix':
        pd_tb_1, pd_tb_2 = pd.read_csv(data_path[0]), pd.read_csv(data_path[1])
        pd_patient_index_1, pd_patient_index_2 = pd_tb_1[pd_tb_1.columns[0]]+'_1', pd_tb_2[pd_tb_2.columns[0]]+'_2'
        pd_patient_value_1, pd_patient_value_2 = pd_tb_1[pd_tb_1.columns[1::]], pd_tb_2[pd_tb_2.columns[1::]]
        pd_patient_value_1, col_1 = feature_filter(pd_patient_value_1, load_path = feature_selection_df_savepath, outcome_name = outcome_name, data_type_ = 'mean', )
        pd_patient_value_2, col_2 = feature_filter(pd_patient_value_2, load_path = feature_selection_df_savepath, outcome_name = outcome_name, data_type_ = 'var', )
        pd_patient_value_mix = pd.DataFrame(np.concatenate((pd_patient_value_1, pd_patient_value_2), axis=1), columns=col_1+col_2)
        return col_1+col_2, pd_patient_value_mix

    else:
        pd_tb = pd.read_csv(data_path)
        pd_patient_index = pd_tb[pd_tb.columns[0]]
        pd_patient_value = pd_tb[pd_tb.columns[1::]]
        return pd_patient_index, pd_patient_value

In [5]:
grid =False

grid_stack = []
result_dict={'outcome':[], 'model':[], 'Reg:mae(train)':[], 'Reg:mae(valid)': [], 'Reg:mse(train)':[], 'Reg:mse(valid)': [], 'Reg:mape(train)':[], 'Reg:mape(valid)': [], }
def fxn():
    warnings.warn("deprecated", DeprecationWarning)
if True:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        fxn()
        # classification : AD8 [0-1, >=2]{0,1}, MOCA [<=23, >=24]{0,1}
        # outcome_name = 'MOCA客觀認知測驗分數[<=23,>=24]' #AD8主觀認知障礙[0-1,>=2], MOCA客觀認知測驗分數[<=23,>=24]
        model_stack = model_list(grid=grid)
        kfold = KFold(n_splits=4)
        for outcome_name in ["AD8主觀認知障礙[0-1,>=2]", "MOCA客觀認知測驗分數[<=23,>=24]"]:
            for model in model_stack:
                if model in ['MLPR', 'XBGR', 'LGBMR']:
                    result_dict['outcome'].append(outcome_name)
                    result_dict['model'].append(model)
                    label_ = clinical_data[outcome_name]
                    if 'MOCA' in outcome_name:
                        label_s = [1 if (i>=24)  else 0  for i in list(label_)]
                    elif 'AD8' in outcome_name:
                        label_s = [1 if i>=2 else 0 for i in list(label_)]
                        label_ = label_+1
                    if data_type!='mix':
                        _, patient_value = loader_(data_path = os.path.join(raw_data_path,next_path))
                    else:
                        _, patient_value = loader_(data_path = [os.path.join(raw_data_path,next_path[0]),os.path.join(raw_data_path,next_path[1])], outcome_name=outcome_name)
                    # patientlse:
                    #     print("_value_index = clinical_data['subj']
                    X_train, X_test, Y_label, _  = train_test_split(patient_value, list(label_s) , stratify=list(label_s), random_state=123)
                    y_train, y_test, _, _ = train_test_split(list(label_), list(label_s) , stratify=list(label_s), random_state=123)
                    X_train, X_valid, _, _ = train_test_split(X_train, list(Y_label) , stratify=list(Y_label), test_size=0.5, random_state=123)
                    y_train, y_valid, _, _ = train_test_split(y_train, list(Y_label) , stratify=list(Y_label), test_size=0.5,random_state=123)
                    print("train: ",X_train.shape)
                    print("valid: ",X_valid.shape)
                    print("Feature Number: {}".format(X_train.shape[1]))
                    if grid==False:
                        print("ALFF Outcome: {}[{}] - cut rank point: {} - GridSearch: {}".format(outcome_name, data_type, rank_threshold, grid))
                        print("--Start Training {} model--".format(model))
                        if model in 'MLPR':
                            model = model_stack[model].fit(X_train, y_train)
                        else:
                            
                            model = model_stack[model].fit(np.array(X_train), np.array(y_train), eval_set=[[np.array(X_valid), np.array(y_valid)]], eval_metric="mape", verbose=False)
                        train_pred =  model.predict(X_train)
                        
                        valid_pred =  model.predict(X_valid)
                    else:
                        print("ALFF Outcome: {}[{}] - cut rank point: {} - GridSearch: {}".format(outcome_name, data_type, rank_threshold, grid))
                        print("--Start Training {} model--".format(model))
                        GridModel = GridSearchCV(model_stack[model][0],model_stack[model][1], cv=kfold, verbose=-1, refit = 'neg_mean_absolute_percentage_error', scoring="neg_mean_absolute_percentage_error",
                                                 return_train_score=True)
                        GridModel.fit(np.array(X_train).astype(np.float16), np.array(y_train).astype(np.int8))
                        param_grid = model_stack[model][1]
                        print('Best parameter : [{}]\n'.format(GridModel.best_params_))
                        train_pred =  GridModel.predict(X_train)
                        valid_pred =  GridModel.predict(X_valid)
                        print("Data Length. Training : {} || Validation: {}".format(len(y_train), len(y_valid)))
                        print("pass",model)
                        grid_stack.append(GridModel)
                    
                    result_dict['Reg:mae(train)'].append(round(metrics.mean_absolute_error(y_train,train_pred), 5))
                    result_dict['Reg:mae(valid)'].append(round(metrics.mean_absolute_error(y_valid,valid_pred), 5))
                    result_dict['Reg:mse(train)'].append(round(metrics.mean_squared_error(y_train,train_pred),5))
                    result_dict['Reg:mse(valid)'].append(round(metrics.mean_squared_error(y_valid,valid_pred),5))
                    result_dict['Reg:mape(train)'].append(round(metrics.mean_absolute_percentage_error(y_train,train_pred),5))
                    result_dict['Reg:mape(valid)'].append(round(metrics.mean_absolute_percentage_error(y_valid,valid_pred),5))

                    # print('Train MAE: ', round(metrics.mean_absolute_error(y_train,train_pred), 5),' || ','MSE: ',round(metrics.mean_squared_error(y_train,train_pred),5),' || ','MAPE: ',round(metrics.mean_absolute_percentage_error(y_train,train_pred),5))
                    # print('Valid MAE: ', round(metrics.mean_absolute_error(y_valid,test_pred), 5),' || ','MSE: ',round(metrics.mean_squared_error(y_valid,test_pred),5),' || ','MAPE: ',round(metrics.mean_absolute_percentage_error(y_valid,test_pred),5))
                    # print('Test MAE: ', round(metrics.mean_absolute_error(y_test,test_pred), 5),' || ','MSE: ',round(metrics.mean_squared_error(y_test,test_pred),5),' || ','MAPE: ',round(metrics.mean_absolute_percentage_error(y_test,test_pred),5))
                    print('----------------------------------------------------')
                # print('\n')
                

train:  (31, 16)
valid:  (32, 16)
Feature Number: 16
ALFF Outcome: AD8主觀認知障礙[0-1,>=2][mix] - cut rank point: 8 - GridSearch: False
--Start Training MLPR model--
----------------------------------------------------
train:  (31, 16)
valid:  (32, 16)
Feature Number: 16
ALFF Outcome: AD8主觀認知障礙[0-1,>=2][mix] - cut rank point: 8 - GridSearch: False
--Start Training XBGR model--
----------------------------------------------------
train:  (31, 16)
valid:  (32, 16)
Feature Number: 16
ALFF Outcome: AD8主觀認知障礙[0-1,>=2][mix] - cut rank point: 8 - GridSearch: False
--Start Training LGBMR model--
----------------------------------------------------
train:  (31, 14)
valid:  (32, 14)
Feature Number: 14
ALFF Outcome: MOCA客觀認知測驗分數[<=23,>=24][mix] - cut rank point: 8 - GridSearch: False
--Start Training MLPR model--
----------------------------------------------------
train:  (31, 14)
valid:  (32, 14)
Feature Number: 14
ALFF Outcome: MOCA客觀認知測驗分數[<=23,>=24][mix] - cut rank point: 8 - GridSearch: False
--

In [6]:
for i in grid_stack:
    print(i.best_params_)
    print('----------------------------------------------------------------------------------------------------------------------------------------')

In [7]:
result_dict

{'outcome': ['AD8主觀認知障礙[0-1,>=2]',
  'AD8主觀認知障礙[0-1,>=2]',
  'AD8主觀認知障礙[0-1,>=2]',
  'MOCA客觀認知測驗分數[<=23,>=24]',
  'MOCA客觀認知測驗分數[<=23,>=24]',
  'MOCA客觀認知測驗分數[<=23,>=24]'],
 'model': ['MLPR', 'XBGR', 'LGBMR', 'MLPR', 'XBGR', 'LGBMR'],
 'Reg:mae(train)': [1.79531, 0.00034, 1.85848, 3.12931, 0.00033, 3.07804],
 'Reg:mae(valid)': [1.73157, 1.55122, 1.83569, 3.06267, 2.92333, 2.75],
 'Reg:mse(train)': [5.95239, 0.0, 5.27367, 13.44134, 0.0, 12.88866],
 'Reg:mse(valid)': [5.08062, 3.8377, 4.64871, 13.93395, 13.23919, 10.47626],
 'Reg:mape(train)': [0.71219, 0.0002, 1.03724, 0.12458, 1e-05, 0.12699],
 'Reg:mape(valid)': [0.69168, 0.71052, 1.02063, 0.12112, 0.1195, 0.11344]}

In [8]:
result_df = pd.DataFrame(result_dict) 
result_df.style.set_caption("AD8/MOCA - [ALFF - {} - Grid: {}]".format(data_type, grid))

,outcome,model,Reg:mae(train),Reg:mae(valid),Reg:mse(train),Reg:mse(valid),Reg:mape(train),Reg:mape(valid)
0,"AD8主觀認知障礙[0-1,>=2]",MLPR,1.795310,1.731570,5.952390,5.080620,0.712190,0.691680
1,"AD8主觀認知障礙[0-1,>=2]",XBGR,0.000340,1.551220,0.000000,3.837700,0.000200,0.710520
2,"AD8主觀認知障礙[0-1,>=2]",LGBMR,1.858480,1.835690,5.273670,4.648710,1.037240,1.020630
3,"MOCA客觀認知測驗分數[<=23,>=24]",MLPR,3.129310,3.062670,13.441340,13.933950,0.124580,0.121120
4,"MOCA客觀認知測驗分數[<=23,>=24]",XBGR,0.000330,2.923330,0.000000,13.239190,0.000010,0.119500
5,"MOCA客觀認知測驗分數[<=23,>=24]",LGBMR,3.078040,2.750000,12.888660,10.476260,0.126990,0.113440
